In [1]:
%time
%load_ext autotime
%load_ext autoreload
%autoreload 2


# if cannot import the modules, add the parent directory to system path might help

import os, tqdm, sys
parent_dir = os.path.abspath(os.getcwd()+'/..')+'/'
sys.path.append(parent_dir)

from utils.save import make_dir, save_pickle, load_pickle, save
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import re
import tqdm
random_seed = 2019

data = load_pickle('../big_data/recipe1M_cleaned.pickle')

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 11 µs


### old version

In [20]:
def reverse(x): return x
def txt(v, fields, mode = 'train'):
    '''
    fields: an order list, the last is the field to predict
    mode: test/train, return string X, y or X+y
    '''
    to_write = ''
    for field in fields:
        if field == 'title':
            name = reverse(v['title'])
            to_write += ' <start-title>'+name+' <end-title>'
        if field == 'ingredients':
            ingredients = [reverse(sent) for sent in v['ingredients']]
            to_write += ' <start-ingredients>'+'$'.join(ingredients)+'$ <end-ingredients>'
        if field == 'directions':
            directions = [reverse(sent) for sent in v['instructions']]
            to_write += ' <start-directions>'+' '.join(directions)+' <end-directions>'
                                                     
    if mode == 'train':
        return to_write
                                                     
    elif mode == 'test':
        field_to_predict = '<start-%s>'%fields[-1]
        to_X, to_y = to_write.split(field_to_predict)
        return to_X+field_to_predict, to_y

class to_gpt2:
    def __init__(self, data, ls):
        ls_train, self.ls_test, _, __ = train_test_split(ls, ls, 
                                                         test_size = 0.2, 
                                                         random_state = random_seed, 
                                                         shuffle = True)
        
        self.ls_train, self.ls_val, _, __ = train_test_split(ls_train, 
                                                             ls_train, 
                                                             test_size = 0.25, 
                                                             random_state = random_seed , 
                                                             shuffle = True)
        self.data = data
        
    def train(self, ls, filename, overwrite = False, is_val = False):
        to_write = ''
        for i, v in enumerate(self.data):
            if i in ls:      
                self.save(filename+'%d'%(i)+'d.txt', txt(v, ['title','ingredients','directions']), overwrite)
                self.save(filename+'%d'%(i)+'i.txt', txt(v, ['title','directions','ingredients']), overwrite)
                self.save(filename+'%d'%(i)+'t.txt', txt(v, ['ingredients','directions','title']), overwrite)
                
    def train_reduce(self, ls, filename, overwrite = False, is_val = False):
        to_write = ''
        for i, v in enumerate(self.data):
            if i in ls:      
                self.save(filename+'%d'%(i)+'d.txt', txt(v, ['title','ingredients','directions']), overwrite)
                
    def test(self, ls, filename, overwrite = False, is_val = False):
        to_write = ''
        for i, v in enumerate(self.data):
            if i in ls:
                self.save(filename+'X/%d'%(i)+'d.txt', txt(v, ['title','ingredients','directions'], mode = 'test')[0], overwrite)
                self.save(filename+'X/%d'%(i)+'i.txt', txt(v, ['title','directions','ingredients'], mode = 'test')[0], overwrite)
                self.save(filename+'X/%d'%(i)+'t.txt', txt(v, ['ingredients','directions','title'], mode = 'test')[0], overwrite)
                
                self.save(filename+'y/%d'%(i)+'d.txt', txt(v, ['title','ingredients','directions'], mode = 'test')[1], overwrite)
                self.save(filename+'y/%d'%(i)+'i.txt', txt(v, ['title','directions','ingredients'], mode = 'test')[1], overwrite)
                self.save(filename+'y/%d'%(i)+'t.txt', txt(v, ['ingredients','directions','title'], mode = 'test')[1], overwrite)
                                
    def fast_export(self, filename, overwrite = False):
        prefix = 'sample_train/'
        self.train_reduce(self.ls_train[:500], filename+prefix, overwrite = overwrite)
        prefix = 'sample_val/'
        self.train_reduce(self.ls_val[:50], filename+prefix,overwrite = overwrite)
        prefix = 'sample_test/'
        self.test(self.ls_test[:50], filename+prefix, overwrite = overwrite)
        
        prefix = 'train/'
        self.train_reduce(self.ls_train, filename+prefix, overwrite = overwrite)
        prefix = 'val/'
        self.train_reduce(self.ls_val, filename+prefix,overwrite = overwrite)
        prefix = 'test/'
        self.test(self.ls_test, filename+prefix, overwrite = overwrite)
        
        prefix = 'train_500/'
        self.test(self.ls_train[:500], filename+prefix, overwrite = overwrite)
        prefix = 'test_500/'
        self.test(self.ls_test[:500], filename+prefix, overwrite = overwrite)
        
    def save(self, filename, to_write, overwrite = False):
        make_dir(filename)
        if os.path.isfile(filename) == True and overwrite == False:
            print('already exists'+filename)
        else:    
            with open(filename,'w') as f:
                f.write('%s' % to_write)

time: 68.9 ms


### export a smaller version

In [27]:
ls = range(len(data[:54000]))

time: 24.3 ms


In [28]:
model = to_gpt2(data[:54000], ls)

time: 63.6 ms


In [30]:
model.fast_export('../../to_gpt2/recipe54k_1010s/',overwrite = True)

make dir
make dir
make dir
make dir
make dir
make dir
make dir
make dir
make dir
make dir
make dir
make dir
time: 3.98 s


In [39]:
ls = range(100)
y, x, _, __ = train_test_split(ls, ls, test_size = 0.2, random_state = random_seed, shuffle = True)

time: 31.4 ms


### export and a full version

In [ ]:
ls = range(len(data))
model = to_gpt2(data, ls)
model.fast_export('../../to_gpt2/recipe54k_1010/',overwrite = True)

### Make data loading faster

In [2]:
cd /data/yueliu/RecipeAnalytics_201906/

/data/yueliu/RecipeAnalytics_201906
time: 17.5 ms


In [3]:
import os, importlib
save = importlib.import_module("gpt-2.src.save")
to_write = "path = '/data/yueliu/RecipeAnalytics_201906/gpt-2/'"+'\n'+"path_to_model = path + 'models/'"
save.save('gpt-2/src/path.py', to_write, overwrite = True)

saved gpt-2/src/path.py
time: 29.5 ms


In [4]:
cd gpt-2

/data/yueliu/RecipeAnalytics_201906/gpt-2
time: 27.3 ms


In [5]:
from src import encoder

time: 50.4 ms


In [6]:
enc = encoder.get_encoder('117M')

time: 106 ms


In [7]:
def reverse(x): return x
def txt(v, fields, mode = 'train'):
    '''
    fields: an order list, the last is the field to predict
    mode: test/train, return string X, y or X+y
    '''
    to_write = ''
    for field in fields:
        if field == 'title':
            name = reverse(v['title'])
            to_write += ' <start-title>'+name+' <end-title>'
        if field == 'ingredients':
            ingredients = [reverse(sent) for sent in v['ingredients']]
            to_write += ' <start-ingredients>'+'$'.join(ingredients)+'$ <end-ingredients>'
        if field == 'directions':
            directions = [reverse(sent) for sent in v['instructions']]
            to_write += ' <start-directions>'+' '.join(directions)+' <end-directions>'
            
    to_write= to_write.replace('..','.')
                                                     
    if mode == 'train':
        return to_write
                                                     
    elif mode == 'test':
        field_to_predict = '<start-%s>'%fields[-1]
        to_X, to_y = to_write.split(field_to_predict)
        return to_X+field_to_predict, to_y

import random

class to_gpt2:
    def __init__(self, data, ls = None):
        if not ls:
            ls = list(range(len(data)))
        random.seed(random_seed)
        random.shuffle(ls)
        self.ls_test = ls[:10000] 
        self.ls_val = ls[10000:20000]
        self.ls_train = ls[20000:]
        self.data = data
        
    def train(self, ls, filename, overwrite = False, is_val = False):
        to_write = ''
        for i, v in tqdm.tqdm(enumerate(self.data)):
            if i in ls:      
                self.save(filename+'%d'%(i)+'d.txt', txt(v, ['title','ingredients','directions']), overwrite)
                self.save(filename+'%d'%(i)+'i.txt', txt(v, ['title','directions','ingredients']), overwrite)
                self.save(filename+'%d'%(i)+'t.txt', txt(v, ['ingredients','directions','title']), overwrite)
                
    def train_reduce(self, ls, filename, overwrite = False, is_val = False):
        to_write = ''
        for i, v in tqdm.tqdm(enumerate(self.data)):
            if i in ls:      
                self.save(filename+'%d'%(i)+'d.txt', txt(v, ['title','ingredients','directions']), overwrite)
                
    def test(self, ls, filename, overwrite = False, is_val = False):
        to_write = ''
        for i, v in tqdm.tqdm(enumerate(self.data)):
            if i in ls:
                self.save(filename+'X/%d'%(i)+'d.txt', txt(v, ['title','ingredients','directions'], mode = 'test')[0], overwrite)
                self.save(filename+'X/%d'%(i)+'i.txt', txt(v, ['title','directions','ingredients'], mode = 'test')[0], overwrite)
                self.save(filename+'X/%d'%(i)+'t.txt', txt(v, ['ingredients','directions','title'], mode = 'test')[0], overwrite)
                
                self.save(filename+'y/%d'%(i)+'d.txt', txt(v, ['title','ingredients','directions'], mode = 'test')[1], overwrite)
                self.save(filename+'y/%d'%(i)+'i.txt', txt(v, ['title','directions','ingredients'], mode = 'test')[1], overwrite)
                self.save(filename+'y/%d'%(i)+'t.txt', txt(v, ['ingredients','directions','title'], mode = 'test')[1], overwrite)
        
    def save(self, filename, to_write, overwrite = False):
        make_dir(filename)
        if os.path.isfile(filename) == True and overwrite == False:
            print('already exists'+filename)
        else:    
            with open(filename,'w') as f:
                f.write('%s' % to_write)
                
    def make_chunk(self, ls, filename, tag, overwrite = False):
        chunk = []
        for i, v in tqdm.tqdm(enumerate(self.data)):
            if i in ls:      
                chunk.append(self.encode_recipe(self.data[i]))
        make_dir(filename)
        print(filename+tag)
        save_pickle(filename+tag, chunk)
        
    def encode_recipe(self,recipe):
        return enc.encode(txt(recipe, ['title','ingredients','directions']))
    
    def fast_chunk(self, filename, overwrite = False):
        self.make_chunk(self.ls_train, filename, tag = 'chunk.train')
        self.make_chunk(self.ls_val, filename, tag = 'chunk.val')
        self.make_chunk(self.ls_test, filename, tag = 'chunk.test')
        
    def fast_export(self, filename, overwrite = False):
        prefix = 'sample_train/'
        self.train_reduce(self.ls_train[:500], filename+prefix, overwrite = overwrite)
        prefix = 'sample_val/'
        self.train_reduce(self.ls_val[:50], filename+prefix,overwrite = overwrite)
        prefix = 'sample_test/'
        self.test(self.ls_test[:50], filename+prefix, overwrite = overwrite)
        
        prefix = 'test/'
        self.test(self.ls_test, filename+prefix, overwrite = overwrite)
        
        prefix = 'train_500/'
        self.test(self.ls_train[:500], filename+prefix, overwrite = overwrite)
        prefix = 'test_500/'
        self.test(self.ls_test[:500], filename+prefix, overwrite = overwrite)

time: 64.7 ms


In [8]:
pwd

'/data/yueliu/RecipeAnalytics_201906/gpt-2'

time: 48.7 ms


In [9]:
model = to_gpt2(data)
model.fast_chunk('../to_gpt2/recipe1M_1010/', overwrite = True)
model.fast_export('../to_gpt2/recipe1M_1010/', overwrite = True)

1029720it [9:16:37, 30.83it/s]


make dir
../to_gpt2/recipe54k_1010/chunk.train


1029720it [04:36, 3724.34it/s]
423it [00:00, 4222.95it/s]

../to_gpt2/recipe54k_1010/chunk.val


1029720it [04:34, 3754.54it/s]
12330it [00:00, 123290.71it/s]

...

make dir
make dir


1029720it [00:09, 105602.90it/s]
25398it [00:00, 127010.61it/s]

make dir
make dir


1029720it [00:09, 107800.36it/s]

time: 9h 31min 1s
